# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Loaded the csv exported in Part I to a DataFrame

In [2]:
weather_map_csv = "../WeatherPy/City_Weather.csv"
weather_map_df = pd.DataFrame(pd.read_csv(weather_map_csv))
weather_map_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Ushuaia,-54.8000,-68.3000,AR,1619810784,41.00,81,75,1.16
1,Avarua,-21.2078,-159.7750,CK,1619810787,78.80,83,75,4.61
2,Mahébourg,-20.4081,57.7000,MU,1619811042,77.00,83,20,11.50
3,Malbork,54.0359,19.0266,PL,1619811043,42.01,100,100,6.06
4,Margate,51.3813,1.3862,GB,1619810862,46.00,65,38,1.01


### Humidity Heatmap
* Configured gmaps.
* Used the Lat and Lng as locations and Humidity as the weight.
* Added Heatmap layer to map.

In [3]:
#Use API gkey to access info about maps 
gmaps.configure(api_key = g_key)

In [4]:
#Identify locations as lat and lng/humidity as weight -- convert to float to fill NaN values
locations = weather_map_df[["Latitude","Longitude"]]
humidity = weather_map_df["Humidity"].astype(float)

#Plot Google Heatmap
figure = gmaps.figure(zoom_level = 1.80, center = (20, 5))

#Set max intensity based on highest humidity in the df
max_intensity = weather_map_df["Humidity"].max()

#create and combine heat layer on top of Google map
heat_map_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_intensity, point_radius=3)

figure.add_layer(heat_map_layer)

figure

Figure(layout=FigureLayout(height='420px'))

# Created a new DataFrame fitting weather criteria
* Narrowed down the cities to fit weather conditions.
* Dropped any rows will null values.

In [5]:
vacation_weather_df = weather_map_df.loc[(weather_map_df["Max Temperature"] <80) & (weather_map_df["Max Temperature"]>70) & (weather_map_df["Wind Speed"] <10) & (weather_map_df["Cloudiness"]) == 0, :]

#Drop NaNs
vacation_weather_df.dropna()

#reset index
vacation_weather_df.reset_index(drop=True, inplace = True)

vacation_weather_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Ushuaia,-54.8000,-68.3000,AR,1619810784,41.00,81,75,1.16
1,Mahébourg,-20.4081,57.7000,MU,1619811042,77.00,83,20,11.50
2,Malbork,54.0359,19.0266,PL,1619811043,42.01,100,100,6.06
3,Margate,51.3813,1.3862,GB,1619810862,46.00,65,38,1.01
4,Aklavik,68.2191,-135.0107,CA,1619810841,28.40,80,100,8.05


### Hotel Map
* Stored dataframe into a variable named `hotel_df`.
* Added a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Stored the first Hotel result into the DataFrame.
* Plotted markers on top of the heatmap.

In [6]:
hotel_df = vacation_weather_df[["City", "Latitude", "Longitude", "Country", "Date", "Max Temperature", "Humidity", "Cloudiness", "Wind Speed"]].copy()
hotel_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Ushuaia,-54.8000,-68.3000,AR,1619810784,41.00,81,75,1.16
1,Mahébourg,-20.4081,57.7000,MU,1619811042,77.00,83,20,11.50
2,Malbork,54.0359,19.0266,PL,1619811043,42.01,100,100,6.06
3,Margate,51.3813,1.3862,GB,1619810862,46.00,65,38,1.01
4,Aklavik,68.2191,-135.0107,CA,1619810841,28.40,80,100,8.05


In [7]:
#Add "Hotel Name" Column
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Ushuaia,-54.8000,-68.3000,AR,1619810784,41.00,81,75,1.16,
1,Mahébourg,-20.4081,57.7000,MU,1619811042,77.00,83,20,11.50,
2,Malbork,54.0359,19.0266,PL,1619811043,42.01,100,100,6.06,
3,Margate,51.3813,1.3862,GB,1619810862,46.00,65,38,1.01,
4,Aklavik,68.2191,-135.0107,CA,1619810841,28.40,80,100,8.05,


In [8]:
#set radius for 5000m for nearby hotels
keyword = "hotel"
radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#pass in parameters
params = {"keyword": keyword, "radius": radius, "key": g_key}

In [9]:
#for loop to request city info within 5000m
for index, row in hotel_df[0:10].iterrows():
    
    #add coordinates columns
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    #include into parameters settings
    params["location"] = f"{latitude}, {longitude}"
    
    #Get API request response and results
    response = requests.get(base_url, params = params).json()
    results = response["results"]
    
    #Add try block to prevent code from breaking
    try:
        #add hotel result to df and print it
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
        print(f"{row['City']}: the nearest hotel is {results[0]['name']}.")
    
    except:
        print(f"{row['City']}: hotel information missing, next result...")

print("-------------------------------------------------------")


Ushuaia: the nearest hotel is Arakur Ushuaia Resort & Spa.
Mahébourg: the nearest hotel is Holiday Inn Mauritius Mon Tresor.
Malbork: the nearest hotel is Hotel Centrum Malbork.
Margate: the nearest hotel is The Yarrow Hotel.
Aklavik: the nearest hotel is Sundog Inn.
Sumbe: the nearest hotel is Hotel Kalunda.
Garoowe: the nearest hotel is Cadduun Hotel.
Tura: the nearest hotel is Hotel Polo Orchid.
Albany: hotel information missing, next result...
Kapaa: the nearest hotel is Hotel Coral Reef.
-------------------------------------------------------


In [10]:
#Dataframe displaying 10 results
hotels_reduced_df = hotel_df[0:10].copy()
hotels_reduced_df

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Ushuaia,-54.8000,-68.3000,AR,1619810784,41.00,81,75,1.16,Arakur Ushuaia Resort & Spa
1,Mahébourg,-20.4081,57.7000,MU,1619811042,77.00,83,20,11.50,Holiday Inn Mauritius Mon Tresor
2,Malbork,54.0359,19.0266,PL,1619811043,42.01,100,100,6.06,Hotel Centrum Malbork
3,Margate,51.3813,1.3862,GB,1619810862,46.00,65,38,1.01,The Yarrow Hotel
4,Aklavik,68.2191,-135.0107,CA,1619810841,28.40,80,100,8.05,Sundog Inn
5,Sumbe,-11.2061,13.8437,AO,1619810867,78.75,85,86,4.09,Hotel Kalunda
6,Garoowe,8.4054,48.4845,SO,1619811047,74.66,83,100,9.64,Cadduun Hotel
7,Tura,25.5198,90.2201,IN,1619810837,79.41,38,32,2.39,Hotel Polo Orchid
8,Albany,42.6001,-73.9662,US,1619810647,53.01,69,100,8.99,
9,Kapaa,22.0752,-159.3190,US,1619810769,75.20,73,75,13.80,Hotel Coral Reef


In [11]:
#Drop any null info and update df
hotels_reduced_df.dropna()
hotels_reduced_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Ushuaia,-54.8000,-68.3000,AR,1619810784,41.00,81,75,1.16,Arakur Ushuaia Resort & Spa
1,Mahébourg,-20.4081,57.7000,MU,1619811042,77.00,83,20,11.50,Holiday Inn Mauritius Mon Tresor
2,Malbork,54.0359,19.0266,PL,1619811043,42.01,100,100,6.06,Hotel Centrum Malbork
3,Margate,51.3813,1.3862,GB,1619810862,46.00,65,38,1.01,The Yarrow Hotel
4,Aklavik,68.2191,-135.0107,CA,1619810841,28.40,80,100,8.05,Sundog Inn


In [12]:
hotels_reduced_df.isnull().sum()

City               0
Latitude           0
Longitude          0
Country            0
Date               0
Max Temperature    0
Humidity           0
Cloudiness         0
Wind Speed         0
Hotel Name         0
dtype: int64

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
print(locations)

     Latitude  Longitude
0    -54.8000   -68.3000
1    -20.4081    57.7000
2     54.0359    19.0266
3     51.3813     1.3862
4     68.2191  -135.0107
..        ...        ...
522   60.7180    11.1942
523  -20.3297   -40.2925
524   48.9413   -68.6460
525   25.0209   118.2898
526   18.9048    -3.5265

[527 rows x 2 columns]


In [14]:
# Add marker layer ontop of heat map
pin_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#Plot second heat map just like original map
heatmap_2 = gmaps.figure(zoom_level = 2, center = (20,5))

figure.add_layer(pin_markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))